In [13]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Set random seed for reproducibility
tf.random.set_seed(42)

# Define parameters
IMG_SIZE = (180, 180)
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5


In [6]:
# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'Training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 4100 images belonging to 5 classes.
Found 1022 images belonging to 5 classes.


In [7]:
# Test data generator (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'Testing_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 1275 images belonging to 5 classes.


In [22]:

# Load pre-trained DenseNet model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(180, 180, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)


Epoch 1/15
128/128 [==============================] - 91s 625ms/step - loss: 1.1524 - accuracy: 0.5809 - val_loss: 1.1178 - val_accuracy: 0.5806
Epoch 2/15
128/128 [==============================] - 39s 305ms/step - loss: 0.9691 - accuracy: 0.6170 - val_loss: 1.1567 - val_accuracy: 0.5444
Epoch 3/15
128/128 [==============================] - 41s 321ms/step - loss: 0.9238 - accuracy: 0.6325 - val_loss: 1.1954 - val_accuracy: 0.5796
Epoch 4/15
128/128 [==============================] - 42s 326ms/step - loss: 0.9188 - accuracy: 0.6377 - val_loss: 1.0574 - val_accuracy: 0.5857
Epoch 5/15
128/128 [==============================] - 41s 319ms/step - loss: 0.8694 - accuracy: 0.6448 - val_loss: 1.0485 - val_accuracy: 0.5917
Epoch 6/15
128/128 [==============================] - 40s 315ms/step - loss: 0.8473 - accuracy: 0.6625 - val_loss: 1.0346 - val_accuracy: 0.5645
Epoch 7/15
128/128 [==============================] - 41s 321ms/step - loss: 0.8294 - accuracy: 0.6713 - val_loss: 1.1034 - val_ac

In [23]:
base_model.trainable = True

# Freeze the first few layers (optional, but can help prevent overfitting)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=30
)

Epoch 1/30
128/128 [==============================] - 66s 420ms/step - loss: 2.4555 - accuracy: 0.4484 - val_loss: 1.4503 - val_accuracy: 0.5232
Epoch 2/30
128/128 [==============================] - 46s 358ms/step - loss: 1.2350 - accuracy: 0.6064 - val_loss: 1.3797 - val_accuracy: 0.5403
Epoch 3/30
128/128 [==============================] - 45s 351ms/step - loss: 1.0038 - accuracy: 0.6460 - val_loss: 1.3049 - val_accuracy: 0.5625
Epoch 4/30
128/128 [==============================] - 46s 358ms/step - loss: 0.8910 - accuracy: 0.6588 - val_loss: 1.1736 - val_accuracy: 0.5796
Epoch 5/30
128/128 [==============================] - 46s 356ms/step - loss: 0.8370 - accuracy: 0.6745 - val_loss: 1.1449 - val_accuracy: 0.5806
Epoch 6/30
128/128 [==============================] - 46s 355ms/step - loss: 0.7862 - accuracy: 0.6831 - val_loss: 1.0744 - val_accuracy: 0.6048
Epoch 7/30
128/128 [==============================] - 46s 360ms/step - loss: 0.7524 - accuracy: 0.7016 - val_loss: 0.9936 - val_ac

In [15]:
model_base = tf.keras.models.load_model('tomato_classifier_tf.h5')

In [ ]:
# adding more layers:
model = tf.keras.models.Sequential([
    layers.Input(shape=(180, 180, 3)),
    model_base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(5, activation='softmax')  
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# import tensorflow as tf
# from tensorflow.keras import layers, models
# from tensorflow.keras.applications import EfficientNetB0

# # Load the base model with pre-trained weights
# base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# # Set the base model layers to be trainable or frozen as needed
# base_model.trainable = True

# # Optionally freeze the first few layers to prevent overfitting
# for layer in base_model.layers[:100]:
#     layer.trainable = False

# # Create a new model on top of the base model
# model = models.Sequential([
#     layers.Input(shape=(224, 224, 3)),
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dropout(0.2),
#     layers.Dense(256, activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(0.2),
#     layers.Dense(5, activation='softmax')  # 5 classes for tomato classification
# ])

# # Compile the model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # Summary of the model
# model.summary()


In [24]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f"Test accuracy: {test_accuracy:.4f}")


39/39 [==============================] - 7s 161ms/step - loss: 0.5975 - accuracy: 0.7692
Test accuracy: 0.7692


In [2]:
# Save the model
base_model.save('tomato_classifier_tf.h5')


NameError: name 'base_model' is not defined

In [ ]:
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('tomato_classifier_tf.h5')


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])  


history = model.fit(x_train, y_train,  # Replace with your actual training data
                    epochs=10,  # Number of additional epochs
                    validation_data=(x_val, y_val))  # Replace with your actual validation data

# Save the updated model (optional)
model.save('tomato_classifier_tf_updated.h5')


In [14]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f"Test accuracy: {test_accuracy:.4f}")


39/39 [==============================] - 6s 156ms/step - loss: 0.6102 - accuracy: 0.7628
Test accuracy: 0.7628


In [ ]:
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=5
)